In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finance-final/finance.csv


In [2]:
df = pd.read_csv("../input/finance-final/finance.csv")
# df_csv = pd.read_csv("/kaggle/input/finance/finance.csv")

In [3]:
df.head(10)

,Date,Company,Total_Revenue,Net_Income,Total_Assets,Total_Liabilities,Cash_Flow_Operating_Activities,Filing_Type
0,2024-09-30,Microsoft,65585,24667,523013,115200,34180,Q-10
1,2024-06-30,Microsoft,245122,88136,512163,243686,118548,K-10
2,2024-03-31,Microsoft,61858,21939,484275,231123,31917,Q-10
3,2023-12-31,Microsoft,62020,21870,470558,232290,18853,Q-10
4,2023-09-30,Microsoft,56517,22291,445785,225071,30583,Q-10
5,2023-06-30,Microsoft,211915,72361,411976,205753,87582,K-10
6,2023-03-31,Microsoft,52857,18299,380088,185405,24441,Q-10
7,2022-12-31,Microsoft,52747,16425,364552,181416,11173,Q-10
8,2022-09-30,Microsoft,50122,17556,359784,186218,23198,Q-10
9,2022-06-30,Microsoft,198270,72738,364840,198298,89035,K-10


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Date                            39 non-null     object
 1   Company                         39 non-null     object
 2   Total_Revenue                   39 non-null     int64 
 3   Net_Income                      39 non-null     int64 
 4   Total_Assets                    39 non-null     int64 
 5   Total_Liabilities               39 non-null     int64 
 6   Cash_Flow_Operating_Activities  39 non-null     int64 
 7   Filing_Type                     39 non-null     object
dtypes: int64(5), object(3)
memory usage: 2.6+ KB


In [5]:
# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
# Step 1: Filter 10-K filings and get last three fiscal years for each company
k10_df = df[df['Filing_Type'] == 'K-10'].sort_values(['Company', 'Date'], ascending=[True, False])
last_three_k10 = k10_df.groupby('Company').head(3).sort_values(['Company', 'Date'])

In [7]:
# Step 2: Calculate YoY percentage changes for financial metrics
financial_metrics = ['Total_Revenue', 'Net_Income', 'Total_Assets', 'Total_Liabilities', 'Cash_Flow_Operating_Activities']

In [8]:
# Ensure data is sorted chronologically for accurate pct_change
last_three_k10 = last_three_k10.sort_values(['Company', 'Date'])
for metric in financial_metrics:
    last_three_k10[f'{metric}_YoY'] = last_three_k10.groupby('Company')[metric].pct_change() * 100

In [9]:
# Step 3: Additional aggregations
# Aggregate mean YoY changes per company
yoy_aggregate = last_three_k10.groupby('Company')[['Total_Revenue_YoY', 'Net_Income_YoY']].mean().reset_index()


In [10]:
# Total assets and liabilities over the three years
asset_liability = last_three_k10.groupby('Company')[['Total_Assets', 'Total_Liabilities']].agg(['mean', 'max', 'min'])

In [11]:
# Step 4: Summary
# Display key results
print("Last Three 10-K Filings with YoY Changes:")
display(last_three_k10)

Last Three 10-K Filings with YoY Changes:


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Date,Company,Total_Revenue,Net_Income,Total_Assets,Total_Liabilities,Cash_Flow_Operating_Activities,Filing_Type,Total_Revenue_YoY,Net_Income_YoY,Total_Assets_YoY,Total_Liabilities_YoY,Cash_Flow_Operating_Activities_YoY
34,2022-09-24,Apple,394328,99803,352755,302083,122151,K-10,NaN,NaN,NaN,NaN,NaN
30,2023-09-30,Apple,383285,96995,352583,290437,110543,K-10,-2.800461,-2.813543,-0.048759,-3.855232,-9.502992
26,2024-09-28,Apple,391305,93736,364980,308030,118254,K-10,2.092438,-3.359967,3.516052,6.057424,6.975566
9,2022-06-30,Microsoft,198270,72738,364840,198298,89035,K-10,NaN,NaN,NaN,NaN,NaN
5,2023-06-30,Microsoft,211915,72361,411976,205753,87582,K-10,6.882030,-0.518299,12.919636,3.759493,-1.631942
1,2024-06-30,Microsoft,245122,88136,512163,243686,118548,K-10,15.669962,21.800417,24.318650,18.436183,35.356580
24,2021-12-31,Tesla,53823,5644,62131,30548,11497,K-10,NaN,NaN,NaN,NaN,NaN
20,2022-12-31,Tesla,81462,12587,82338,36440,14724,K-10,51.351653,123.015592,32.523217,19.287678,28.068192
16,2023-12-31,Tesla,96773,14974,106618,43009,13256,K-10,18.795267,18.964010,29.488207,18.026894,-9.970117


In [12]:
print("\nAverage YoY Growth Rates (%):")
display(yoy_aggregate)



Average YoY Growth Rates (%):


,Company,Total_Revenue_YoY,Net_Income_YoY
0,Apple,-0.354011,-3.086755
1,Microsoft,11.275996,10.641059
2,Tesla,35.073460,70.989801


In [13]:
print("\nAsset and Liability Analysis:")
display(asset_liability)


Asset and Liability Analysis:


Total_Assets                 Total_Liabilities                
                    mean     max     min              mean     max     min
Company                                                                   
Apple      356772.666667  364980  352583     300183.333333  308030  290437
Microsoft  429659.666667  512163  364840     215912.333333  243686  198298
Tesla       83695.666667  106618   62131      36665.666667   43009   30548